MOTIVAÇÃO
---------

O óleo diesel é muito importante no Brasil, pois é A FONTE DE ENERGIA PARA um dos principais modais de transporte no Brasil, que é o rodoviário. É tão importante que em 2018 uma alta dos preços resultou em uma greve de caminhoneiros, paralisando boa parte dos transportes e causando uma série de transtornos não somente na indústria e comércio, mas também no fornecimento de itens básicos para a população.

O valor do combustível tem relação direta com o preço da maioria dos demais produtos devido ao frete. Sendo assim tem sido um fator importante para inflação no país.

Temos 2 tipos de óleo diesel:
- Óleo diesel S10 contém menor teor de enxofre, sendo menos poluente e adequado para veículos produzidos após 2012.

- Óleo diesel comum, também conhecido como S500.

Este Kernel analisa a variação de preço do combustível final (diesel) ao consumidor, comparando com a variação do óleo bruto no mercado internacional e especulando possíveis razões para o resultado.

Referências : 
[Petrobrás](http://www.petrobras.com.br/pt/produtos-e-servicos/produtos/automotivos/oleo-diesel/)

Autor : Mauro Vianna

Data  : 16/08/2019

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

import seaborn as sns # seaborn package
import matplotlib.pyplot as plt # matplotlib library


In [ ]:
# abre dataset com preços de combustíveis
os.chdir('/kaggle/input/gas-prices-in-brazil/')
df = pd.read_csv('2004-2019.tsv', sep='\t',parse_dates=[1,2])
df.sample(5)


LIMPEZA DE DADOS
----------------

In [ ]:
# verifica tipos de campo
df.info()

In [ ]:
# Remoção de coluna indesejada
df2 = df.drop("Unnamed: 0", axis=1)

# Conversão de colunas numéricas que constam como object para os tipos adequados
col_numeros = ['UNIDADE DE MEDIDA', 'MARGEM MÉDIA REVENDA', 'PREÇO MÉDIO DISTRIBUIÇÃO', 
            'DESVIO PADRÃO DISTRIBUIÇÃO', 'PREÇO MÍNIMO DISTRIBUIÇÃO', 'PREÇO MÁXIMO DISTRIBUIÇÃO',
            'COEF DE VARIAÇÃO DISTRIBUIÇÃO']
for col in col_numeros:
    df2[col] = pd.to_numeric(df2[col], errors='coerce')
df2.dtypes

In [ ]:
# Lista produtos
produtos = df2['PRODUTO'].unique()
for p in produtos: print(p)

In [ ]:
# Identifica faixa de tempo
def ano_mes(data):  # formata para mês-ano
    am = str(data%100) + "-"+ str(data//100)
    return am

meses = df2['MÊS'] + df2['ANO']*100 
meses = meses.unique()
meses.sort()

print(ano_mes(meses[0]) + ' até '+ ano_mes(meses[-1]))


In [ ]:
# Verifica valores Null
df2.info()

In [ ]:
df2.sample(5)

In [ ]:
# Carrega dataset de preços de óleo bruto
tea_oil = pd.read_csv('/kaggle/input/tea-and-oil-price-data-gem-database/export_dt.csv', index_col=0, parse_dates=True)
tea_oil['ANO'] = tea_oil.index.year
tea_oil['MÊS'] = tea_oil.index.month
tea_oil.head()


ANÁLISE
-------

In [ ]:
#cor = 'white' # eu uso um notebook com fundo escuro. Para fundo claro, remova o comentário abaixo
cor = 'black'

def muda_cor(graf,titulo):  # muda cores e título para um gráfico
    graf.xaxis.label.set_color(cor)
    graf.xaxis.label.set_color(cor)
    graf.tick_params(axis='x', colors=cor)
    graf.tick_params(axis='y', colors=cor)
    graf.set_title(titulo, color=cor)

    graf.grid(True)
    return 

# plot Regionwise Yearwise Average Price data
#plt.figsize=(39,15)
fig, axes  = plt.subplots(2, 2, sharey=True,figsize=(15,13))

df2.query('2012<=ANO<=2018 & PRODUTO in ["ÓLEO DIESEL"]').groupby(['ANO', 'REGIÃO'])['PREÇO MÉDIO REVENDA'].agg('mean').unstack().plot(ax=axes[0,0])
df2.query('2012<=ANO<=2018 & PRODUTO in ["ÓLEO DIESEL S10"]').groupby(['ANO', 'REGIÃO'])['PREÇO MÉDIO REVENDA'].agg('mean').unstack().plot(ax=axes[0,1])
df2.query('ANO!=2019 & PRODUTO in ["ÓLEO DIESEL"]').groupby(['ANO', 'REGIÃO'])['PREÇO MÉDIO REVENDA'].agg('mean').unstack().plot(ax=axes[1,0])
df2.query('ANO!=2019 & PRODUTO in ["ÓLEO DIESEL S10"]').groupby(['ANO'])['PREÇO MÉDIO REVENDA'].agg('mean').plot(ax=axes[1,1])
df2.query('ANO!=2019 & PRODUTO in ["ÓLEO DIESEL"]').groupby(['ANO'])['PREÇO MÉDIO REVENDA'].agg('mean').plot(ax=axes[1,1])

#configura cada gráfico
muda_cor(axes[0,0], 'ÓLEO DIESEL 2012-2018')
muda_cor(axes[0,1], 'ÓLEO DIESEL S10 2012-2018')
muda_cor(axes[1,0], 'ÓLEO DIESEL 2004-2018')
muda_cor(axes[1,1], 'ÓLEO DIESEL S10 x ÓLEO DIESEL 2004-2018')

#plt.grid(True)
#ax1.grid(True)
#for axis in axes: axis.grid(True)
#axes

Alguns pontos observados nos gráficos:

- Temos um custo maior nas regiões Norte e Centro-Oeste. Uma possível explicação seria o custo de logística para transportar o combustível para estas regiões. Dependemos muito da importação de diesel, que é feita através de meio marítimo. Os principais portos de entrada são os de Paranaguá, Santos e São Luiz;
- A região sul tem o menor valor. Provavelmente devido ao porto de Paranaguá;
- O diesel S10 é um pouco mais caro que o S500;
- No período de 2006 a 2012 tivemos preço sem maiores oscilações. A partir de 2012, tivemos uma subida de preços

Referências:
* [FGV - artigo de 2016 sobre dependência de importação](https://fgvenergia.fgv.br/noticias/brasil-continua-dependente-do-diesel-de-fora)

* [Artigo da FATEC sobre impacto da Lava Jato na importação de Diesel](https://even3.blob.core.windows.net/anais/90151.pdf)

In [ ]:
fig, ax1  = plt.subplots(figsize=(15,7))


df2.query('2004<=ANO<=2018 & PRODUTO in ["ÓLEO DIESEL"]').groupby(['ANO'])['PREÇO MÉDIO REVENDA'].agg('mean').plot(ax=ax1, color ='green')
ax2 = ax1.twinx()
df2.query('2004<=ANO<=2018 & PRODUTO in ["ÓLEO DIESEL"]').groupby(['ANO'])['PREÇO MÉDIO DISTRIBUIÇÃO'].agg('mean').plot(ax=ax1, color ='blue')
ax2 = ax1.twinx()
tea_oil.query('2004<=ANO<=2018').groupby(['ANO'])['OIL'].agg('mean').plot(ax=ax2, color ='red')


muda_cor(ax1, 'DIESEL x ÓLEO BRUTO')
muda_cor(ax2, '')

No gráfico acima comparamos os valores de preço do Diesel (preço final para o consumidor e preço de distribuição em reais) com o preço do óleo bruto em dólares, conforme o World Bank's Global Economic Monitor (GEM). Estamos comparando  produtos diferentes e em moedas dierentes, logo fatores como câmbio e inflação foram descartados. 
Um dos objetivos desta comparação é avaliar o comportamento do preço do diesel no Brasil. 
Era de se esperar alguma correlação entre estes dois produtos (matéria-prima e produto final). Mas isso não acontece. No Brasil, este mercado é fortemente influenciado pela Petrobrás (e subsidiárias), empresa estatal que praticamente detém o monopólio de extração, refino e distribuição de derivados de petróleo.

Podemos observar as seguintes questões:

* O preço final do diesel acompanha o preço de distribuição;
* O preço do diesel não segue o preço do óleo bruto. É possível notar que variações bruscas no valor do óleo bruto, positivas ou negativas, implicam em aumento do diesel;
* O preço do diesel não variou muito até 2012, apesar de variações expressivas do valor do óleo bruto;
* Após 2012 o preço do diesel dispara, apresentando grande crescimento anual mesmo após queda vertiginosa do óleo bruto em 2015. 

Especulações :
* A estabilização do preço do diesel até 2012 provavelmente foi devida a influência governamental na Petrobrás. Tal fato é inclusivereforçado  no site do partido do governo da época;
* Devido ao fato de importarmos diesel e a Petrobrás controlar tanto extração quanto a distribuição, ela está sujeita a queda de receita com grandes variações do óleo bruto, sejam elas positivas ou negativas;
* Eventos como o Petrolão (escândalo de corrupção na Petrobrás descoberto durante a operação Lava-Jato) fragilizaram muito a Petrobrás, o que também implicou na alta dos preço, devido a falta de capacidade de "absorver" os impactos das variações internacionais;
* Como em 2019 a Petrobrás está vendendo a distribuição, é possível que no futuro tenhamos aumento de concorrência e preços mais corentes com os internacionais.


Sugestões de  melhorias nesta análise:
* considerar variação cambial
* comparar preços de diesel no mercado internacional

A adicionar:

* mapas com regiões
* mapas com estados
* posição dos principais portos no mapa
* comparação de preço do diesel com cesta básica
* análise de combustíveis renováveis (etanol, biodiesel)